### Trading Framework Usage Guide

This notebook demonstrates how to use the trading framework built in the `CryptoKit` file. We will import the classes and methods to manage data retrieval, API connections, and order execution.

---

#### Setup

First, ensure you have the required dependencies installed:

```bash
pip install ccxt alpaca-trade-api pandas
```

Create a `config.json` file in the same directory with your API credentials:

```json
{
    "binance": {
        "key": "your-binance-api-key",
        "secret": "your-binance-secret-key"
    },
    "cryptocom": {
        "key": "your-cryptocom-api-key",
        "secret": "your-cryptocom-secret-key"
    },
    "alpaca": {
        "key": "your-alpaca-api-key",
        "secret": "your-alpaca-secret-key"
    }
}
```

---

## **1. Initialization**

In [30]:
import time
from crypto_kit import Logger, Listener, AlpacaOrderManager, OrderType, APIKey
from datetime import datetime

# Load configuration
import json
with open("config.json", "r") as config_file:
    config = json.load(config_file)

In [31]:
# Initialize the Logger
logger = Logger(db_path="example_logs.db")

# Initialize AlpacaOrderManager with API keys
alpaca_credentials = APIKey(
    key=config.get("alpaca", {}).get("key", ""), 
    secret=config.get("alpaca", {}).get("secret", "")
)
order_manager = AlpacaOrderManager(
    exchange_id="alpaca", 
    credentials=alpaca_credentials,
    logger=logger
)

# Initialize Listener for live prices
listener = Listener(
    exchange_id="binance",
    crypto_symbols=["BTC/USDT", "ETH/USDT"],
    logger=logger
)

print("Initialization complete.")

Initialization complete.


## **2. Logging Demonstration**

The `Logger` class logs messages to both the console and a SQLite database. Let's add an example log entry.

In [32]:
logger.log("Starting the CryptoKit Example Notebook")
print("Log entry added. Check `example_logs.db` for details.")

2024-12-23 17:03:06,038 - Starting the CryptoKit Example Notebook


Log entry added. Check `example_logs.db` for details.


## **3. Fetching Live Prices**

The `Listener` class can fetch live prices of cryptocurrencies. Below, we fetch the live price of `BTC/USDT`.

In [33]:
print("Fetching live price of BTC/USDT:")
price = listener.fetch_price("BTC/USDT")
if price:
    print(f"Live price of {price.symbol}: {price.price}")
else:
    print("Failed to fetch live price.")

Fetching live price of BTC/USDT:


2024-12-23 17:03:06,293 - Live price of BTC/USDT: 93363.6


Live price of BTC/USDT: 93363.6


## **4. Continuous Live Price Updates**

We can listen to continuous updates of cryptocurrency prices. This example demonstrates five price updates.

In [34]:
print("Starting listener for live prices:")
try:
    for _ in range(5):  # Limit to 5 
        price = listener.fetch_price("BTC/USDT")
        if price:
            print(f"Live price update: {price.symbol} is {price.price}")
        else:
            print("Failed to fetch live price update.")
        time.sleep(1)  
except KeyboardInterrupt:
    print("Listener stopped.")

Starting listener for live prices:


2024-12-23 17:03:06,557 - Live price of BTC/USDT: 93345.51


Live price update: BTC/USDT is 93345.51


2024-12-23 17:03:07,833 - Live price of BTC/USDT: 93371.54


Live price update: BTC/USDT is 93371.54


...

Live price update: BTC/USDT is 93403.95


2024-12-23 17:03:10,367 - Live price of BTC/USDT: 93403.94


Live price update: BTC/USDT is 93403.94


2024-12-23 17:03:11,627 - Live price of BTC/USDT: 93414.62


Live price update: BTC/USDT is 93414.62


## **5. Fetching Historical Data**

Retrieve historical OHLCV data for a specific cryptocurrency and timeframe. This example fetches daily data for `BTC/USDT` from January 1, 2023, to June 1, 2023.

In [35]:
import pandas as pd

In [36]:
start_date = datetime(2024, 12, 23)
# end date now
end_date = datetime.now()


print("Fetching historical data for BTC/USDT:")
historical_data = listener.fetch_historical_data("BTC/USDT", "4h", start_date, end_date)
if historical_data:
    print(f"Fetched {len(historical_data)} records.")
    print("Sample data:")
    print(historical_data[:5])  
else:
    print("Failed to fetch historical data.")

historical_data_df = pd.DataFrame(historical_data)
historical_data_df.style.background_gradient(cmap='viridis')

Fetching historical data for BTC/USDT:


2024-12-23 17:03:12,912 - Fetched batch of historical data for BTC/USDT with timeframe 4h.


Fetched 5 records.
Sample data:
[{'timestamp': datetime.datetime(2024, 12, 23, 1, 0), 'open': 95186.28, 'high': 96043.99, 'low': 93700.42, 'close': 96025.15, 'volume': 6028.4424}, {'timestamp': datetime.datetime(2024, 12, 23, 5, 0), 'open': 96025.15, 'high': 96509.98, 'low': 94705.07, 'close': 95404.01, 'volume': 3919.1348}, {'timestamp': datetime.datetime(2024, 12, 23, 9, 0), 'open': 95404.01, 'high': 96538.92, 'low': 95036.77, 'close': 95932.42, 'volume': 3020.85272}, {'timestamp': datetime.datetime(2024, 12, 23, 13, 0), 'open': 95932.41, 'high': 96423.0, 'low': 93107.52, 'close': 93250.37, 'volume': 8316.98998}, {'timestamp': datetime.datetime(2024, 12, 23, 17, 0), 'open': 93250.38, 'high': 93428.29, 'low': 93201.38, 'close': 93428.29, 'volume': 110.84968}]


,timestamp,open,high,low,close,volume
0,2024-12-23 01:00:00,95186.280000,96043.990000,93700.420000,96025.150000,6028.442400
1,2024-12-23 05:00:00,96025.150000,96509.980000,94705.070000,95404.010000,3919.134800
2,2024-12-23 09:00:00,95404.010000,96538.920000,95036.770000,95932.420000,3020.852720
3,2024-12-23 13:00:00,95932.410000,96423.000000,93107.520000,93250.370000,8316.989980
4,2024-12-23 17:00:00,93250.380000,93428.290000,93201.380000,93428.290000,110.849680


## **6. Creating and Canceling an Order**

Using the `AlpacaOrderManager`, we create a market order for `BTC/USD`. After a short delay, we attempt to cancel the order.

In [37]:
print("Creating and attempting to cancel an order:")
order = order_manager.create_order(
    symbol="BTC/USD",
    order_type=OrderType.MARKET,
    side="buy",
    qty=0.001
)
if order:
    print("Order created successfully.")
    time.sleep(5)  # Delay to ensure the order is processed
    result = order_manager.cancel_order(order_id=getattr(order, 'id', None))
    if result:
        print("Order canceled successfully.")
    else:
        print("Order cancellation failed. It might already be filled.")
else:
    print("Failed to create order.")

Creating and attempting to cancel an order:


2024-12-23 17:03:13,624 - Order placed: Order({   'asset_class': 'crypto',
    'asset_id': '276e2673-764b-4ab6-a611-caf665ca6340',
    'canceled_at': None,
    'client_order_id': '38197b18-1d05-4f70-9319-f25ae6575e14',
    'created_at': '2024-12-23T16:03:13.577990353Z',
    'expired_at': None,
    'expires_at': '2025-03-23T20:00:00Z',
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': '5ebad919-1b6e-4fed-b725-4c3c7c97c5ba',
    'legs': None,
    'limit_price': None,
    'notional': None,
    'order_class': '',
    'order_type': 'market',
    'position_intent': 'buy_to_open',
    'qty': '0.001',
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': 'buy',
    'source': None,
    'status': 'pending_new',
    'stop_price': None,
    'submitted_at': '2024-12-23T16:03:13.577990353Z',
    'subtag': None,
    'symbol': 'BTC/USD',
    'time_in_force': 'gtc',
   

Order created successfully.


2024-12-23 17:03:18,763 - Error canceling order: order is already in "filled" state


Order cancellation failed. It might already be filled.


## **7. Managing Positions**

Retrieve and manage open positions. If a position in `BTC/USD` exists, attempt to exit it.

In [38]:
print("Fetching positions and exiting BTC/USD position:")
positions = order_manager.fetch_positions()
if positions:
    for position in positions:
        symbol = getattr(position, 'symbol', None)
        if symbol == "BTCUSD":
            print(f"Exiting position for {symbol}.")
            result = order_manager.exit_position(symbol)
            if result:
                print(f"Position for {symbol} exited successfully.")
            else:
                print(f"Failed to exit position for {symbol}.")
else:
    print("No positions to exit.")

2024-12-23 17:03:18,904 - Fetched positions: [Position({   'asset_class': 'crypto',
    'asset_id': '64bbff51-59d6-4b3c-9351-13ad85e3c752',
    'asset_marginable': False,
    'avg_entry_price': '93325.975',
    'change_today': '-0.0131652817553077',
    'cost_basis': '93.120658',
    'current_price': '93337.5',
    'exchange': 'CRYPTO',
    'lastday_price': '94582.708',
    'market_value': '93.132158',
    'qty': '0.0009978',
    'qty_available': '0.0009978',
    'side': 'long',
    'symbol': 'BTCUSD',
    'unrealized_intraday_pl': '0.0115',
    'unrealized_intraday_plpc': '0.0001234956909347',
    'unrealized_pl': '0.0115',
    'unrealized_plpc': '0.0001234956909347'})]


Fetching positions and exiting BTC/USD position:
Exiting position for BTCUSD.


2024-12-23 17:03:19,027 - Exiting position: 0.0009978 BTCUSD with a sell order
2024-12-23 17:03:19,156 - Order placed: Order({   'asset_class': 'crypto',
    'asset_id': '276e2673-764b-4ab6-a611-caf665ca6340',
    'canceled_at': None,
    'client_order_id': '0ee3b8a6-749c-475a-89f3-5b8c16b6e645',
    'created_at': '2024-12-23T16:03:19.110026956Z',
    'expired_at': None,
    'expires_at': '2025-03-23T20:00:00Z',
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'hwm': None,
    'id': '86584c81-0b53-488b-a80f-233109b7422e',
    'legs': None,
    'limit_price': None,
    'notional': None,
    'order_class': '',
    'order_type': 'market',
    'position_intent': 'sell_to_close',
    'qty': '0.0009978',
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': 'sell',
    'source': None,
    'status': 'pending_new',
    'stop_price': None,
    'submitted_at': '2024-12-23T16:03:19.110

Position for BTCUSD exited successfully.


## **8. Fetching Open Orders**

Check for any open orders using the `fetch_open_orders` method.

In [39]:
print("Fetching open orders:")
open_orders = order_manager.fetch_open_orders()
if open_orders:
    print(f"Found {len(open_orders)} open orders.")
else:
    print("No open orders found.")

2024-12-23 17:03:19,286 - Open orders: []


Fetching open orders:
No open orders found.


## **9. Wrap-Up**

This concludes the demonstration of the CryptoKit functionalities. Check the logs for detailed information on all operations.

In [40]:
logger.log("Finished executing the CryptoKit Example Notebook")

2024-12-23 17:03:19,300 - Finished executing the CryptoKit Example Notebook


All examples executed successfully.


In [29]:
import sqlite3
import pandas as pd

def fetch_all_logs(db_path: str) -> pd.DataFrame:
    """
    Fetch all logs from the database and return as a pandas DataFrame.
    """
    conn = sqlite3.connect(db_path)
    query = "SELECT * FROM logs ORDER BY timestamp DESC"
    logs_df = pd.read_sql_query(query, conn)
    conn.close()
    return logs_df

# Fetch and display all logs
print("Fetching all logs from the database:")
logs_df = fetch_all_logs("example_logs.db")
if not logs_df.empty:
    print(logs_df.head())  # Display the first few logs
else:
    print("No logs found.")

# Optionally, display the logs in a more detailed way
for _, row in logs_df.iterrows():
    print(f"[{row['timestamp']}] {row['message']}")

Fetching all logs from the database:
   id            timestamp                                            message
0  24  2024-12-23 16:00:22  Finished executing the CryptoKit Example Notebook
1  23  2024-12-23 16:00:03                                    Open orders: []
2  20  2024-12-23 15:59:19  Fetched positions: [Position({   'asset_class'...
3  21  2024-12-23 15:59:19  Exiting position: 0.0009978 BTCUSD with a sell...
4  22  2024-12-23 15:59:19  Order placed: Order({   'asset_class': 'crypto...
[2024-12-23 16:00:22] Finished executing the CryptoKit Example Notebook
[2024-12-23 16:00:03] Open orders: []
[2024-12-23 15:59:19] Fetched positions: [Position({   'asset_class': 'crypto',
    'asset_id': '64bbff51-59d6-4b3c-9351-13ad85e3c752',
    'asset_marginable': False,
    'avg_entry_price': '93063.75',
    'change_today': '-0.0145874233163212',
    'cost_basis': '92.85901',
    'current_price': '93202.99',
    'exchange': 'CRYPTO',
    'lastday_price': '94582.708',
    'market_value